## **Cálculo de las emisiones de CO2 emitidas por la generación de electricidad en Florida**

### Esta API calcula la emisión de CO2 para algunas actividades comunes. Es de libre acceso

### Para este propósito se utilizará la REST-API que puede ser consulada en: 
#### [Carbon interface](https://docs.carboninterface.com/#/)


In [185]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
key = "KDPebxwd6cjRYVYycS6vA"
status_codes={
    200:"OK",
    201:"Object created",
    400:"Bad request",
    401:"Unauthorized",
    403:"Forbidden",
    404:"Not found",
    422:"Unprocessable entity",
    429:"Too many request",
    500:"Internal Server Error",
    503:"Service Unavailable"
        }


In [3]:
def acceso(key):
    url="https://www.carboninterface.com/api/v1/auth"
    authorization=requests.get(url,headers={"Authorization": "Bearer " + key})
    if authorization.status_code == 200:
        return print("Connection authorized")     
    else:
        return print(f"Connection failed, status code: {authorization.status_code} \nMeaning: {status_codes[authorization.status_code]}")

In [4]:
acceso(key)

Connection authorized


In [5]:
def unidad_electricidad():

    unidad=input("Introduce las unidades en las que la energía será medida. Las opciones" +
                    " válidas son: \n- mwh \n-kwh\n\n"+
                    "Para salir, presione s")
    unidad=unidad.lower()
    
    if unidad == "s":
        print("Hasta luego")
        return 
        
    elif (unidad != "kwh" and unidad != "mwh"): 
        print("El valor introducido no coincide con ninguna opción válida, intente de nuevo.")
        return unidad_electricidad()
    

In [6]:
def valor_electricidad():
    try:
        valor=float(input("Introduce la cantidad de energía eléctrica utilizada:"))
        return valor
    except:
        print("El valor introducido no es válido, intente de nuevo")
        return valor_electricidad()

In [7]:
def electricidad():
    
    type="electricity"
    unidad=unidad_electricidad()
    valor=valor_electricidad()
    country="us"
    
    header={
        "Content-Type":"application/json",
        "Authorization": "Bearer " + key,
    }
    
    data={
        "type":type,
        "electricity_unit":str(unidad),
        "electricity_value":valor,
        "country":country,
        "state":"fl"
    }
    
    url="https://www.carboninterface.com/api/v1/estimates"
    r=requests.post(url,headers=header,data=json.dumps(data))
    d=pd.DataFrame(json.loads(r.text)) #return a DataFrame, but the first row we have 
    # the information we need
    return pd.json_normalize(d.loc[d.index[0]].data)
    


In [8]:
data = electricidad()
data

,country,state,electricity_unit,electricity_value,estimated_at,carbon_g,carbon_lb,carbon_kg,carbon_mt
0,us,fl,None,20.0,2023-05-24T02:42:08.675Z,7957369,17543.0,7957.37,7.96


In [9]:
print("En EE.UU. en el estado de Florida, la cantidad de CO2 emitida en toneladas son: ",
      data["carbon_mt"][0], 
      "\nla cual fue estimada en ",
      data["estimated_at"][0])

En EE.UU. en el estado de Florida, la cantidad de CO2 emitida en toneladas son:  7.96 
la cual fue estimada en  2023-05-24T02:42:08.675Z


In [164]:
url="https://www.thewindpower.net/statistics_countries_es.php"
code=requests.get(url)
soup=BeautifulSoup(code.text,"html5lib")

In [165]:
capacidad_prod=pd.read_html(str(soup.find_all("table")))[4]

In [166]:
capacidad_prod.head()

,0,1,2,3,4,5,6,7,8,9,10
0,País,2022 2022,2021 2021,2020 2020,2019 2019,2018 2018,2017 2017,2016 2016,2015 2015,2014 2014,2013 2013
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alemania,66 315.0 66 315.0,63 760.0 63 760.0,62 784.0 62 784.0,61 357.0 61 357.0,59 560.0 59 560.0,56 190.0 56 190.0,50 019.0 50 019.0,45 192.0 45 192.0,40 468.0 40 468.0,34 660.0 34 660.0


In [167]:
capacidad_prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       134 non-null    object
 1   1       134 non-null    object
 2   2       134 non-null    object
 3   3       134 non-null    object
 4   4       134 non-null    object
 5   5       134 non-null    object
 6   6       134 non-null    object
 7   7       134 non-null    object
 8   8       134 non-null    object
 9   9       134 non-null    object
 10  10      134 non-null    object
dtypes: object(11)
memory usage: 23.1+ KB


In [168]:
capacidad_prod.dropna(inplace=True)
cols=capacidad_prod.iloc[0,:].tolist()
new_cols=[]
for col in cols:
    new_cols.append(col[:4])

capacidad_prod.columns=new_cols

In [169]:
capacidad_prod.head()
capacidad_prod.drop(0,inplace=True)

In [170]:
capacidad_prod.reset_index(drop=True,inplace=True)

In [171]:
capacidad_prod.head()

,País,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,Albania,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0
1,Alemania,66 315.0 66 315.0,63 760.0 63 760.0,62 784.0 62 784.0,61 357.0 61 357.0,59 560.0 59 560.0,56 190.0 56 190.0,50 019.0 50 019.0,45 192.0 45 192.0,40 468.0 40 468.0,34 660.0 34 660.0
2,Arabia Saudita,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,0.0 0.0,0.0 0.0,0.0 0.0,0.0 0.0
3,Argelia,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2
4,Argentina,3 309.0 3 309.0,3 292.0 3 292.0,2 624.0 2 624.0,1 604.0 1 604.0,640.0 640.0,318.0 318.0,279.0 279.0,279.0 279.0,271.0 271.0,217.0 217.0


In [179]:
table_copy=capacidad_prod.copy()
table_copy.head()

,País,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,Albania,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0,42.0 42.0
1,Alemania,66 315.0 66 315.0,63 760.0 63 760.0,62 784.0 62 784.0,61 357.0 61 357.0,59 560.0 59 560.0,56 190.0 56 190.0,50 019.0 50 019.0,45 192.0 45 192.0,40 468.0 40 468.0,34 660.0 34 660.0
2,Arabia Saudita,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,2.8 2.8,0.0 0.0,0.0 0.0,0.0 0.0,0.0 0.0
3,Argelia,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2,10.2 10.2
4,Argentina,3 309.0 3 309.0,3 292.0 3 292.0,2 624.0 2 624.0,1 604.0 1 604.0,640.0 640.0,318.0 318.0,279.0 279.0,279.0 279.0,271.0 271.0,217.0 217.0


In [184]:
table_copy=capacidad_prod.copy()
for col in table_copy.columns:
    new_cols=[]
    if col != "País":
        icol=table_copy[col] # columna por columna
        list_values=icol.str.replace(r"\xa0"," ", regex=True).str.split(" ").tolist()
        for list_value in list_values: # ya 
            if len(list_value)==2:
                new_cols.append(float(list_value[0]))
            else:
                new_cols.append(float(list_value[0]+list_value[1]))
        table_copy.loc[:,col]=new_cols


table_copy.head()
                
                
            
        


C:\Users\Emmanuel\AppData\Local\Temp\ipykernel_11952\870344604.py:12: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  table_copy.loc[:,col]=new_cols


,País,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,Albania,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0
1,Alemania,66315.0,63760.0,62784.0,61357.0,59560.0,56190.0,50019.0,45192.0,40468.0,34660.0
2,Arabia Saudita,2.8,2.8,2.8,2.8,2.8,2.8,0.0,0.0,0.0,0.0
3,Argelia,10.2,10.2,10.2,10.2,10.2,10.2,10.2,10.2,10.2,10.2
4,Argentina,3309.0,3292.0,2624.0,1604.0,640.0,318.0,279.0,279.0,271.0,217.0


In [246]:
countries_years=table_copy.transpose()
countries_years.head()

,0,1,2,3,4,5,6,7,8,9,...,123,124,125,126,127,128,129,130,131,132
País,Albania,Alemania,Arabia Saudita,Argelia,Argentina,Armenia,Australia,Austria,Azerbaiyán,Bahamas,...,Turquía,Ucrania,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Yibuti,Total
2022,42.0,66315.0,2.8,10.2,3309.0,2.6,10134.0,3736.0,67.0,1.0,...,11396.0,1761.0,0.0,1514.0,0.8,3.9,71.0,4628.0,20.0,899155.7
2021,42.0,63760.0,2.8,10.2,3292.0,2.6,8951.0,3524.0,67.0,1.0,...,10607.0,1761.0,0.0,1514.0,0.8,3.9,71.0,4118.0,20.0,824622.6
2020,42.0,62784.0,2.8,10.2,2624.0,2.6,8603.0,3120.0,66.0,1.0,...,9305.0,1314.0,0.0,1514.0,0.8,3.9,71.0,600.0,0.0,738994.0
2019,42.0,61357.0,2.8,10.2,1604.0,2.6,7133.0,3095.0,66.0,1.0,...,8056.0,1170.0,0.0,1514.0,0.8,3.9,71.0,388.0,0.0,652938.6


In [247]:
new_cols=[]
for col in countries_years.columns:
    new_cols.append(countries_years.loc["País",col])
new_cols
countries_years.columns=new_cols
countries_years.drop("País",inplace=True)
countries_years.head()

,Albania,Alemania,Arabia Saudita,Argelia,Argentina,Armenia,Australia,Austria,Azerbaiyán,Bahamas,...,Turquía,Ucrania,Uganda,Uruguay,Uzbekistán,Vanuatu,Venezuela,Vietnam,Yibuti,Total
2022,42.0,66315.0,2.8,10.2,3309.0,2.6,10134.0,3736.0,67.0,1.0,...,11396.0,1761.0,0.0,1514.0,0.8,3.9,71.0,4628.0,20.0,899155.7
2021,42.0,63760.0,2.8,10.2,3292.0,2.6,8951.0,3524.0,67.0,1.0,...,10607.0,1761.0,0.0,1514.0,0.8,3.9,71.0,4118.0,20.0,824622.6
2020,42.0,62784.0,2.8,10.2,2624.0,2.6,8603.0,3120.0,66.0,1.0,...,9305.0,1314.0,0.0,1514.0,0.8,3.9,71.0,600.0,0.0,738994.0
2019,42.0,61357.0,2.8,10.2,1604.0,2.6,7133.0,3095.0,66.0,1.0,...,8056.0,1170.0,0.0,1514.0,0.8,3.9,71.0,388.0,0.0,652938.6
2018,42.0,59560.0,2.8,10.2,640.0,2.6,5043.0,3045.0,66.0,1.0,...,7154.0,596.0,0.0,1511.0,0.8,3.9,71.0,232.0,0.0,590867.0


In [262]:
top5_df=table_copy.sort_values("2022",ascending=False).head(6)
top5_df

,País,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
132,Total,899155.7,824622.6,738994.0,652938.6,590867.0,539382.5,487302.5,435550.9,371184.9,318825.8
25,China,365964.0,328973.0,281993.0,237029.0,211392.0,187730.0,168730.0,147362.0,114763.0,91324.0
43,Estados Unidos,140862.0,132738.0,122328.0,105433.0,96665.0,89047.0,82033.0,74347.0,65754.0,61108.0
1,Alemania,66315.0,63760.0,62784.0,61357.0,59560.0,56190.0,50019.0,45192.0,40468.0,34660.0
60,India,41930.0,40067.0,38625.0,37529.0,35802.0,32848.0,28279.0,25088.0,22465.0,20150.0
42,España,29308.0,27497.0,27446.0,25808.0,23431.0,23170.0,23075.0,23007.0,22987.0,22959.0


In [297]:
top5=table_copy.sort_values("2022",ascending=False).head(6).País.tolist()
top5.append("México")
top5.remove("Total")
top5.remove("México")
top5

['China', 'Estados Unidos', 'Alemania', 'India', 'España']

In [298]:
plt.rcParams["figure.dpi"] = 1000
plt.figure(figsize=(10,8))
fig=sns.lineplot(data=countries_years[top5].sort_index(ascending=True))
fig.set_title("Top 5 países con la mayor capacidad \nde producción instalada de energía, y México\n(MW)")
fig.set_ylabel("MW")
plt.show()

In [290]:
plt.rcParams["figure.dpi"] = 1000
plt.figure(figsize=(10,8))
fig=sns.lineplot(data=countries_years["México"].sort_index(ascending=True))
fig.set_title("Capacidad de producción eólica en México\n(MW)")
fig.set_ylabel("MW")
plt.show()